# Envelope Selection

In this notebook, we will explore how to select the envelope from information layers.

## 0. Initialization

### 0.1. Load required libraries

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import numpy as np
import pandas as pd

In [ ]:
# extra import function
def lattice_from_csv(file_path):
    # read metadata
    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])
    unit = np.array(meta_df['unit'])
    minbound = np.array(meta_df['minbound'])

    # read lattice
    lattice_df = pd.read_csv(file_path, skiprows=5)

    # create the buffer
    buffer = np.array(lattice_df['value']).reshape(shape)

    # create the lattice
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)

    return l

### 0.2. Load Sun Access Lattice

In [ ]:
# loading the lattice from csv
sun_acc_path = os.path.relpath('../data/sun_access.csv')
sun_acc_lattice = lattice_from_csv(sun_acc_path)

## 1. Envelope Selection

### 1.1. Visualizing the selection

In [ ]:
p = pv.Plotter(notebook=True)

base_lattice = sun_acc_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):

    lattice = np.copy(sun_acc_lattice)
    lattice[sun_acc_lattice < value] *= 0.0
    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F")  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.001, 1.0])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False, clim=[0.0, 1.0])

    return

p.add_slider_widget(create_mesh, [0, 1], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

### 1.2. Generating an envelope based on the selection

In [ ]:
threshold = 0.675
new_avail_lattice = sun_acc_lattice > threshold

### 1.3. Visualize the new available lattice

In [ ]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

### 1.4. Save new envelope to CSV

In [ ]:
csv_path = os.path.relpath('../data/new_envelope_lattice.csv')
new_avail_lattice.to_csv(csv_path)

In [ ]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop [+]"